In [20]:
import pandas as pd
import numpy as np
import math
from scipy import stats

#loading data

df1 = pd.read_csv('imfs1.csv')
imfs1 = df1.values[:,1:]
imfs1 = np.array (imfs1 , dtype= 'float')
df2 = pd.read_csv('imfs2.csv')
imfs2 = df2.values[:,1:]
imfs2 = np.array (imfs2 , dtype= 'float')
df3 = pd.read_csv('imfs3.csv')
imfs3 = df3.values[:,1:]
imfs3 = np.array (imfs3 , dtype= 'float')
dfy = pd.read_csv('y.csv')
y = dfy.values[:,1].reshape(-1,1)
y = np.array (y , dtype= 'float')

#normization

imfs1 = stats.zscore(imfs1)
imfs2 = stats.zscore(imfs2)
imfs3 = stats.zscore(imfs3)

def split_train_test():
    imfs1_train = []  
    imfs1_test =  []
    imfs2_train = []
    imfs2_test = []
    imfs3_train = []
    imfs3_test = []
    y_train = []
    y_test = []
    test_index = [1,5,9,13,17,21,25,30,35,38]
    for i in range(40):
        c = 0
        for j in range(len(test_index)):
            if(i == test_index[j]): 
                c = 1 
                imfs1_test.append(imfs1[i]) 
                imfs2_test.append(imfs2[i])
                imfs3_test.append(imfs3[i])
                y_test.append(y[i])
        if(c == 0):
            imfs1_train.append(imfs1[i]) 
            imfs2_train.append(imfs2[i]) 
            imfs3_train.append(imfs3[i]) 
            y_train.append(y[i])
    imfs1_train = np.array(imfs1_train)
    imfs1_test = np.array(imfs1_test)
    imfs2_train = np.array(imfs2_train)
    imfs2_test = np.array(imfs2_test)
    imfs3_train = np.array(imfs3_train)
    imfs3_test = np.array(imfs3_test)
    y_train = np.array(y_train)
    y_test = np.array(y_test)
    return imfs1_train,imfs1_test,imfs2_train,imfs2_test,imfs3_train,imfs3_test,y_train,y_test


imfs1_train,imfs1_test,imfs2_train,imfs2_test,imfs3_train,imfs3_test,y_train,y_test = split_train_test()

print("y_train.shape:",y_train.shape)
print("imfs1_test.shape:",imfs1_test.shape)

y_train.shape: (30, 1)
imfs1_test.shape: (10, 300)


In [10]:
import tensorflow as tf


In [18]:
learning_rate = 0.001

num_iterations = 300000


x_imf1 = tf.placeholder(shape=[None,300],  dtype='float')
nn = tf.layers.dense(x_imf1, 4, activation=tf.nn.tanh,kernel_regularizer=tf.contrib.layers.l2_regularizer(0.0001))
nn = tf.layers.dense(nn, 6, activation=tf.nn.tanh,kernel_regularizer=tf.contrib.layers.l2_regularizer(0.0001))
nn_1 = tf.layers.dense(nn, 1, activation=tf.nn.tanh,kernel_regularizer=tf.contrib.layers.l2_regularizer(0.0001))

x_imf2 = tf.placeholder(shape=[None, 300],  dtype='float')
nn = tf.layers.dense(x_imf2, 4, activation=tf.nn.tanh,kernel_regularizer=tf.contrib.layers.l2_regularizer(0.0001))
nn = tf.layers.dense(nn, 6, activation=tf.nn.tanh,kernel_regularizer=tf.contrib.layers.l2_regularizer(0.0001))
nn_2 = tf.layers.dense(nn, 1, activation=tf.nn.tanh,kernel_regularizer=tf.contrib.layers.l2_regularizer(0.0001))

x_imf3 = tf.placeholder(shape=[None, 300],   dtype='float')
nn = tf.layers.dense(x_imf3, 4, activation=tf.nn.tanh,kernel_regularizer=tf.contrib.layers.l2_regularizer(0.0001))
nn = tf.layers.dense(nn, 6, activation=tf.nn.tanh,kernel_regularizer=tf.contrib.layers.l2_regularizer(0.0001))
nn_3 = tf.layers.dense(nn, 1, activation=tf.nn.tanh,kernel_regularizer=tf.contrib.layers.l2_regularizer(0.0001))

Y = tf.placeholder(shape=[None, 1],  dtype='float')

fuck = tf.add(nn_1,nn_2)
you = tf.add(fuck,nn_3)

cost = tf.reduce_mean(tf.losses.mean_squared_error(labels=Y,predictions=you))
optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate).minimize(cost)
init = tf.global_variables_initializer()



In [19]:
with tf.Session() as sess:
    sum_cost = 0
    sess.run(init)
    for i in range(num_iterations):
        _ , fuckcost = sess.run([optimizer, cost], feed_dict={x_imf1:imfs1_train , x_imf2:imfs2_train ,x_imf3:imfs3_train  ,Y:y_train})
        sum_cost  += fuckcost
        if ( (i+1) % 10000 == 0):
            print("training cost:",sum_cost/10000)
            sum_cost = 0
    prediction_test = sess.run(you, feed_dict={x_imf1:imfs1_test , x_imf2:imfs2_test ,x_imf3:imfs3_test  ,Y:y_test})
    prediction_train =  sess.run(you, feed_dict={x_imf1:imfs1_train , x_imf2:imfs2_train ,x_imf3:imfs3_train  ,Y:y_train})
    
    print("prediction_train:",prediction_train)
    print("y_train:",y_train)
    sum_se = 0
    for i in range(len(prediction_train)):
        se = float((prediction_train[i] - y_train[i])**2)
        print("se: ",se)
        sum_se += se
    print("mse: ",math.sqrt(sum_se/len(prediction_train)))
    
    print("\n-------------------------------------\n")
    
    print("prediction_test:",prediction_test)
    print("y_test:",y_test)
    sum_se = 0
    for i in range(len(prediction_test)):
        se = float((prediction_test[i] - y_test[i])**2)
        print("se: ",se)
        sum_se += se
    print("mse: ",math.sqrt(sum_se/len(prediction_test)))
            

training cost: 0.004856093240167615
training cost: 1.1439081680801252e-11
training cost: 5.9611856962835105e-12
training cost: 4.682761256102202e-12
training cost: 4.022852548345515e-12
training cost: 3.6026088956215423e-12
training cost: 3.235307917078857e-12
training cost: 2.975159450616452e-12
training cost: 2.7876111541053765e-12
training cost: 2.603033654965427e-12
training cost: 2.4956334055510272e-12
training cost: 2.4005368971921278e-12
training cost: 2.265096441054354e-12
training cost: 2.2003330287997946e-12
training cost: 2.138169185109118e-12
training cost: 2.1057143103405533e-12
training cost: 2.058186118476628e-12
training cost: 2.049345715696149e-12
training cost: 2.0155286444391645e-12
training cost: 1.9435036882576793e-12
training cost: 1.8760439930275848e-12
training cost: 1.8296260722075985e-12
training cost: 1.8525148637316135e-12
training cost: 1.864923611914239e-12
training cost: 1.8129399082878264e-12
training cost: 1.7630578809596856e-12
training cost: 1.7654727